
 ```
 For example, suppose the starting numbers are 0,3,6:

    Turn 1: The 1st number spoken is a starting number, 0.
    Turn 2: The 2nd number spoken is a starting number, 3.
    Turn 3: The 3rd number spoken is a starting number, 6.
    Turn 4: Now, consider the last number spoken, 6. Since that was the first time the number had been spoken, the 4th number spoken is 0.
    Turn 5: Next, again consider the last number spoken, 0. Since it had been spoken before, the next number to speak is the difference between the turn number when it was last spoken (the previous turn, 4) and the turn number of the time it was most recently spoken before then (turn 1). Thus, the 5th number spoken is 4 - 1, 3.
    Turn 6: The last number spoken, 3 had also been spoken before, most recently on turns 5 and 2. So, the 6th number spoken is 5 - 2, 3.
    Turn 7: Since 3 was just spoken twice in a row, and the last two turns are 1 turn apart, the 7th number spoken is 1.
    Turn 8: Since 1 is new, the 8th number spoken is 0.
    Turn 9: 0 was last spoken on turns 8 and 4, so the 9th number spoken is the difference between them, 4.
    Turn 10: 4 is new, so the 10th number spoken is 0.
```

In [4]:
from collections import deque, defaultdict
from typing import List, Dict, Iterator
import itertools

RAW = [0,3,6]

def d():
    return deque(maxlen=2)

def play(snumbers:List[int], limit:int=2020) -> int:
    turn = 1
    track = defaultdict(d)
    snumbers = list(reversed(snumbers))
    while turn <= limit:
        while snumbers:
            num_spoken = snumbers.pop()
            track[num_spoken].appendleft(turn)
            turn += 1
        if len(track[num_spoken]) == 1:
            num_spoken = 0
            track[num_spoken].appendleft(turn)
        else:
            #print(track)
            before = track[num_spoken][-1]
            last = track[num_spoken][0]
            num_spoken = last - before
            track[num_spoken].appendleft(turn)
        turn += 1
    return num_spoken

# inspired by Joel G solution 
def play_iter(snumbers:List[int]) -> Iterator[int]:
    track = defaultdict(d)
    snumbers = deque(snumbers)
    num_spoken = None
    for i in itertools.count(1):
        if snumbers:
            #print('in snumber')
            num_spoken = snumbers.popleft()
            track[num_spoken].appendleft(i)
        elif len(track[num_spoken]) == 1:
            #print('in eq 1')
            num_spoken = 0
            track[num_spoken].appendleft(i)
        else:
            #print("in track")
            before = track[num_spoken][-1]
            last = track[num_spoken][0]
            num_spoken = last - before
            track[num_spoken].appendleft(i)
        yield num_spoken
        
def run_game(numbers: List[int],limit:int=2020)-> int:
    game = play_iter(snumbers=numbers)
    for _ in range(limit):
        n = next(game)
    return n

## Unit Test

In [5]:
assert play(snumbers=RAW) == 436
assert play(snumbers=[1,3,2]) == 1
assert play(snumbers=[2,1,3]) == 10
assert play(snumbers=[1,2,3]) == 27
assert play(snumbers=[2,3,1]) == 78
assert play(snumbers=[3,2,1]) == 438
assert play(snumbers=[3,1,2]) == 1836

In [6]:
assert run_game(numbers=RAW) == 436
assert run_game(numbers=[3,1,2]) == 1836

## Part 1

In [7]:
PUZZLE = [1,12,0,20,8,16]
play(snumbers=PUZZLE)

273

In [8]:
run_game(numbers=PUZZLE)

273

## Part 2

In [9]:
run_game(limit=30000000, numbers=PUZZLE)

47205

In [10]:
play(limit=30000000, snumbers=[1,12,0,20,8,16])

47205

### another solution suggestion

In [11]:
def play_game(starting_numbers: List[int]) -> Iterator[int]:
    last_seen: Dict[int, int] = {}
    gap = None

    for i in itertools.count(0):
        if i < len(starting_numbers):
            # still in starting numbers, use them
            n = starting_numbers[i]
        elif gap:
            # number seen before, so say the gap
            n = gap 
        else:
            # new number, so say zero
            n = 0
        
        if n in last_seen:
            # saw this already, so figure out the gap
            gap = i - last_seen[n]
        else:
            # first time, so no gap
            gap = None

        # update last seen and yield
        last_seen[n] = i
        yield n

def n_limit(starting_numbers: List[int], limit = 2020) -> int:
    game = play_game(starting_numbers)
    for i in range(limit):
        n = next(game)
    return n

In [12]:
assert n_limit([0, 3, 6]) == 436

In [13]:
# great performance- code suggstion!
%time n_limit([2,20,0,4,1,17], limit =30000000)

Wall time: 13.3 s


814

In [14]:
%time play(limit=30000000, snumbers=[2,20,0,4,1,17])

Wall time: 23.7 s


814

In [15]:
# somehow slower then my original solution!
%time run_game(limit=30000000, numbers=[2,20,0,4,1,17])

Wall time: 26.3 s


814